<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#DATAFILE_NAME = "reduced_dim_space_ddG.csv"
DATASET = "LARGE" # "LARGE" or "REDUCED"
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "large_cat_desc_col_names.csv"
  X_COLS = ['3954', '3955', '3957', '3958', '3959', '3960', '3961', '3962', '3963',
       '3964', '3965', '3966', '3967', '3968', '3969', '3970', '3971', '3972',
       '3973']

df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))

In [ ]:
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)
# start with a few randomly selected catalyst
START_SAMPLE_SIZE = 3
random_catalyst_df = df.sample(START_SAMPLE_SIZE)

In [ ]:
original_ee = df['ddG']
def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=100, cd_iterations=10, step_size=0.001):
    if pls_model == None:
        print("Creating a new model. # data items: " + str(len(catalysts)))
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts

new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

In [ ]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=2)
    # fitting the model
    model.fit(X, y)
    return model

from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
pls_large_saved_model

PLSRegression(n_components=3)

In [ ]:
random_catalyst_df[X_COLS]

,3954,3955,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973
Catalyst,,,,,,,,,,,,,,,,,,,
18_6_3,0.751052,0.578241,0.690619,0.648005,-0.341944,-0.750784,-0.698807,-0.726153,1.818622,1.137386,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898
14_6_2,-0.900772,-1.204402,-1.106008,-1.172227,-0.263889,0.546659,0.579985,0.564909,-0.159928,-0.875422,-0.880495,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164
25_2_6,-0.837073,-1.368929,-1.144584,-1.336538,-0.594464,0.205320,0.297249,0.252548,-1.264638,0.474187,0.436071,-0.539398,-0.951100,1.110344,1.555441,1.638297,1.598277,1.521070,1.614430


In [ ]:
new_optimized_catalysts

array([[ 1.75105172,  1.57824082,  1.69061888,  1.6480053 ,  0.65805637,
         0.24921616,  0.30119292,  0.27384715,  2.81862175,  2.13738582,
         2.10437818,  2.78996606,  2.70797219,  0.92137107, -0.41606153,
        -0.34798358, -0.38341459,  1.49218775,  1.69189841],
       [ 0.09922793, -0.20440216, -0.10600771, -0.17222682,  0.73611136,
         1.54665856,  1.57998504,  1.56490863,  0.84007158,  0.12457768,
         0.11950484,  0.39737598,  0.5283841 ,  0.20175791,  0.72229107,
         0.7195551 ,  0.72066245,  0.75390613, -0.15316401],
       [ 0.16292713, -0.36892924, -0.14458353, -0.33653776,  0.40553569,
         1.20531962,  1.29724935,  1.25254827, -0.26463773,  1.47418709,
         1.43607066,  0.46060159,  0.04890047,  2.11034365,  2.55544128,
         2.63829659,  2.59827688,  2.52106958,  2.61442961]])

In [ ]:
# Perform k-nearest neighbors analysis
num_neighbors = 1  # Number of neighbors to consider
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(df[X_COLS])  # Using all the catalysts

NearestNeighbors(n_neighbors=1)

In [ ]:
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

,3954,3955,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973
Catalyst,,,,,,,,,,,,,,,,,,,
18_11_4,0.758129,0.677277,0.686929,0.574038,-0.399013,-0.833174,-0.693664,-0.764237,-0.187753,-0.896976,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898
27_5_2,-0.900772,-1.204402,-1.106008,-1.172227,-0.263889,0.546659,0.579985,0.564909,-0.159928,-0.875422,-0.880495,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164
20_2_3,0.751052,0.578241,0.690619,0.648005,-0.341944,-0.750784,-0.698807,-0.726153,1.818622,1.137386,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898
1_1_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
1_1_5,-0.963382,0.742768,-0.317384,0.281218,2.028871,1.651922,1.575386,1.616959,-0.028326,1.054486,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633
1_1_3,0.751052,0.578241,0.690619,0.648005,-0.341944,-0.750784,-0.698807,-0.726153,1.818622,1.137386,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898
1_10_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
1_10_5,-0.963382,0.742768,-0.317384,0.281218,2.028871,1.651922,1.575386,1.616959,-0.028326,1.054486,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633
1_10_3,0.751052,0.578241,0.690619,0.648005,-0.341944,-0.750784,-0.698807,-0.726153,1.818622,1.137386,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898


In [ ]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
pls_predict_ee(pls_large_saved_model, properties1)

array([-0.78970154])

In [ ]:
random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [ ]:
# We will now do all this in a single loop
highest_ddg_index = df.index[df['ddG'] == df['ddG'].max()]
not_highest_index=[i for i in df.index if i not in highest_ddg_index]
random_catalyst_df = df.loc[not_highest_index].sample(START_SAMPLE_SIZE)
prev_loop_len = len(random_catalyst_df)
while(len(random_catalyst_df) < len(df)):
    new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    if(len(random_catalyst_df) == prev_loop_len):
        print("The processing has blocked and no new catalysts are being added. Exiting.")
        break
    if len(pd.concat([highest_ddg_index.to_series(), random_catalyst_df.index.to_series()], axis=1, join='inner')) > 0:
        print("Found the highest ddG. Exiting.")
        break
    prev_loop_len = len(random_catalyst_df)
    print(prev_loop_len)

6
12
24
48
96
192


In [ ]:
random_catalyst_df.index

Index(['3_4_5', '3_6_1', '19_9_1', '1_1_5', '1_1_1', '1_10_1', '1_10_5',
       '1_2_1', '1_5_1', '1_2_5', '1_9_1', '1_7_1', '1_5_5', '1_8_1', '1_4_1',
       '1_11_5', '10_6_1', '10_3_1', '1_9_5', '10_10_1', '10_4_1', '1_3_5',
       '10_11_1', '1_11_1', '1_6_5', '1_3_1', '12_10_1', '10_6_5', '11_3_1',
       '11_1_1', '10_4_5', '11_6_1', '10_8_1', '10_10_5', '11_10_1', '11_5_1',
       '10_3_5', '10_7_1', '12_11_1', '1_8_5', '10_9_1', '11_2_1', '10_1_5',
       '11_4_1', '14_1_1', '12_10_5', '14_10_1', '14_2_1'],
      dtype='object', name='Catalyst')

In [ ]:
highest_ddg_index = df.index[df['ddG'] ==df['ddG'].max()]
highest_ddg_index

Index(['1_1_5', '1_10_5', '1_11_5', '1_2_5', '1_3_5', '1_4_5', '1_5_5',
       '1_6_5', '1_7_5', '1_8_5',
       ...
       '9_10_5', '9_11_5', '9_2_5', '9_3_5', '9_4_5', '9_5_5', '9_6_5',
       '9_7_5', '9_8_5', '9_9_5'],
      dtype='object', name='Catalyst', length=319)

In [ ]:
type(highest_ddg_index)

pandas.core.indexes.base.Index

In [ ]:
len(pd.concat([highest_ddg_index.to_series(), random_catalyst_df.index.to_series()], axis=1, join='inner'))

16

In [ ]:
not_highest_index=[i for i in df.index if i not in highest_ddg_index]
df.loc[not_highest_index].index

Index(['1_1_1', '1_1_2', '1_1_3', '1_1_4', '1_1_6', '1_10_1', '1_10_2',
       '1_10_3', '1_10_4', '1_10_6',
       ...
       '9_8_1', '9_8_2', '9_8_3', '9_8_4', '9_8_6', '9_9_1', '9_9_2', '9_9_3',
       '9_9_4', '9_9_6'],
      dtype='object', name='Catalyst', length=1584)